# Example 1: Scenario accessibility analysis

First import required libraries. Here we are using model-system data explorer, geopandas and matplotilib.

In [ ]:
import os, sys
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

sys.path.append("../scripts/")
from data_explorer.scenario_data import ScenarioData

Then we'll load scenario data for scenario. Here we load data for 2023. 

Model-system results are stored in same `model-system/Results` folder so we can use relative paths to scenario data.

In [ ]:
scenario1 = ScenarioData(
    name = "2023", 
    result_data_path = "../Results/2023/", 
    base_data_path = "../Base_input_data/2018_zonedata/", 
    scenario_data_path = "../Scenario_input_data/2023/"
)


In [ ]:

import geopandas as gpd
links = gpd.read_file("C:/Users/suppoatt/emme/koko_suomi_network_22-05-2024/koko_suomi.gpkg", engine = "pyogrio", layer = "network_links")
links = links.to_crs("EPSG:3067")


Use subregion of Sourthern Finland

In [ ]:
scenario1.set_subregion("county", ["Uusimaa"])

### Scenario results
Take a look at the data:

Below is accessibility for work tours, when all modes are available for the traveller. Accessibility is defined as expected maximum utility of all mode and destinations that are available for consumer. Plot shows deciles of this accessibility value, so that each bin has equal number of observations.

In [ ]:
# Get accessibility data to geodataframe
data = scenario1.get_result_data("sustainable_accessibility.txt", geometry=True)
column_name = "hb_edu_basic"

# Matplot
fig, ax = plt.subplots(figsize=(12, 8))
plt.style.use("default")

# Discrete colour scale
discrete_cmap = plt.cm.get_cmap("viridis")

# Set bins of 10 quantiles
range = np.quantile(data[column_name], np.arange(0, 1, 0.1))
norm = mpl.colors.BoundaryNorm(range, discrete_cmap.N)  


# Visualize travel times into continuous coloring scheme
data.plot(ax=ax, column=column_name, linewidth=0.03, 
          cmap=discrete_cmap, norm=norm,
          alpha=0.9, legend=True, 
          legend_kwds={"label": "Saavutettavuusdesiilit", "orientation": "vertical"})

# Add links to plot
links[links.layer == 4].plot(ax=ax, edgecolor="white", linestyle="-", linewidth=0.5)
scenario1.get_basemap().plot(ax=ax, facecolor="#add8e6")

# Themes
ax.set_axis_off()
plt.tight_layout()
plt.title(f"Skenaario: {scenario1.name}")

# Set axis bb
xmin, ymin, xmax, ymax = data.total_bounds
pad = 0.05  # add a padding around the geometry
ax.set_xlim(xmin-pad, xmax+pad)
ax.set_ylim(ymin-pad, ymax+pad)